In [1]:
!pip install -U chromadb openai sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.8/787.8 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 24.7 MB/s eta

In [2]:
# === Mount Google Drive to access API key ===
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [3]:
import json

secret_file_path = "/content/drive/My Drive/Secret_Keys/OpenAI_Secret_Key.json"

with open(secret_file_path, "r") as f:
  data=json.load(f)

secret_Key = data["open_ai_key"]

In [8]:
import os
import chromadb
from openai import OpenAI
from chromadb.config import Settings

chroma_client = chromadb.Client(Settings(
        persist_directory = "./chroma_role_memory",
        anonymized_telemetry = False
))

openai_client = OpenAI(api_key = secret_Key)

collections = {
    "planner": chroma_client.get_or_create_collection(name="planner_memory"),
    "assistant": chroma_client.get_or_create_collection(name="assistant_memory"),
    "coordinator": chroma_client.get_or_create_collection(name="coordinator_memory")
}

memory_id_counter = {
    "planner": 0,
    "assistant": 0,
    "coordinator": 0
}

In [26]:
def add_memory(role: str, text: str):
    memory_id = f"{role}_{memory_id_counter[role]}"
    collections[role].add(
        documents=[text],
        ids=[memory_id],
        metadatas=[{"role": role}]
    )
    memory_id_counter[role] += 1
    print(f"Memory added to {role} role.")

In [32]:
def recall_and_respond(role: str, query:str):
    results = collections[role].query(
        query_texts = [query],
        n_results = 3
    )

    context_docs = results.get("documents",[[]])[0]
    context = "\n".join(context_docs) if context_docs else "No relevant memory found"

    prompt = f"You are a helpful {role} in travel planning system. Use the following memory :\n {context}\n\nUser:{query}"

    response = openai_client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {
                "role":"system",
                "content":f"You are a helpful {role} travel assistant",
            },
            {
                "role":"user",
                "content":prompt
            }
        ]
    )

    answer = response.choices[0].message.content.strip()

    print("\n Response from Gpt-4o-mini:")
    print(answer)

In [ ]:
# Console UI
def run():
    print("=== Role/Tool-Specific Memory Scoping ===")
    while True:
        print("\nOptions:")
        print("1. Add Memory")
        print("2. Recall and Ask GPT")
        print("3. Exit")
        choice = input("Choose: ").strip()
        if choice == "1":
            role = input("Enter role (planner / assistant / coordinator): ").strip().lower()
            if role in collections:
                text = input("Enter memory to store: ").strip()
                add_memory(role, text)
            else:
                print("Invalid role.")
        elif choice == "2":
            role = input("Enter role (planner / assistant / coordinator): ").strip().lower()
            if role in collections:
                query = input("Enter your question: ").strip()
                recall_and_respond(role, query)
            else:
                print("Invalid role.")
        elif choice == "3":
            print("Exiting.")
            break
        else:
            print("Invalid choice.")
# Start the interface
run()

=== Role/Tool-Specific Memory Scoping ===

Options:
1. Add Memory
2. Recall and Ask GPT
3. Exit
Memory added to planner role.

Options:
1. Add Memory
2. Recall and Ask GPT
3. Exit

 Response from Gpt-4o-mini:
Based on your preference for beach destinations with a moderate climate in October, here are a few suggestions:

1. **Costa del Sol, Spain** - Known for its beautiful beaches, pleasant weather, and vibrant culture, Costa del Sol enjoys mild temperatures in October, making it perfect for beach activities.

2. **Greek Islands (such as Crete or Rhodes)** - The Greek Islands offer stunning beaches and pleasant weather during October. You'll find fewer crowds, allowing for a more relaxed experience.

3. **Malibu, California** - With its beautiful coastline and moderate temperatures in October, Malibu is great for beachgoers. You can enjoy the ocean and explore local dining options.

4. **Phuket, Thailand** - October marks the transition to the dry season, and while there may be some ra